In [177]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [176]:
data = pd.read_csv('nyc_taxis.csv', delimiter=',', header=0, index_col=0)
data.head(5)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [ ]:
pickup_datetime_update = data['pickup_datetime'].astype('datetime64[ns]')
dropoff_datetime_update = data['dropoff_datetime'].astype('datetime64[ns]')
data['pickup_datetime'] = pickup_datetime_update
data['dropoff_datetime'] = dropoff_datetime_update
pickup_time = data['pickup_datetime'][0]
dropoff_time = data['dropoff_datetime'][0]

def to_seconds(duration):
    return duration.total_seconds()

travel_time =  data['dropoff_datetime'].sub(data['pickup_datetime'], axis = 0)

data['travel_time'] = travel_time

data['travel_time'] = data['travel_time'].apply(to_seconds)

data.head(5)

In [ ]:
# site to find coordinates of places
# https://www.distancesto.com/coordinates/us/empire-state-building-latitude-longitude/history/477798.html

In [ ]:
import reverse_geocoder as rg
import pprint

def reverseGeocode(coordinates):
    result = rg.search(coordinates)
     
    # result is a list containing ordered dictionary.
    pprint.pprint(result)

In [ ]:
# coordinates for Empire State Building
reverseGeocode((40.748541, -73.985758))

In [ ]:
# coordinates for JFK AIRPORT
reverseGeocode((40.647352,  -73.790534))

In [ ]:
# coordinates for NEWARK LIBERTY AIRPORT
reverseGeocode((40.704197,  -74.190124))

In [ ]:
data_JFK = data[abs(data['pickup_latitude'] -  40.748541) <= 0.009999]
data_JFK = data_JFK[abs(data_JFK['pickup_longitude'] + 73.985758) <= 0.009999]
data_JFK = data_JFK[abs(data_JFK['dropoff_latitude'] - 40.647352) <= 0.009999]
data_JFK = data_JFK[abs(data_JFK['dropoff_longitude'] + 73.790534) <= 0.009999]

data_JFK

In [ ]:
times = pd.DatetimeIndex(data_JFK.pickup_datetime)
grouped_by_hour = data_JFK.groupby([times.hour]).mean()/(60*60)
average_trip_time = list(grouped_by_hour['trip_duration'])
pickup_hours = list(range(0, 24))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(pickup_hours, average_trip_time)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Pickup hour',fontsize=15)
plt.ylabel('Average trip duration in hours',fontsize=15)
plt.title('Average trip time from Empire State Building TO JFK Airport against pickup hour',fontsize=15)
plt.show()

In [ ]:
data_Newark = data[abs(data['pickup_latitude'] -  40.748541) <= 0.009999]
data_Newark = data_Newark[abs(data_Newark['pickup_longitude'] + 73.985758) <= 0.009999]
data_Newark = data_Newark[abs(data_Newark['dropoff_latitude'] - 40.704197) <= 0.009999]
data_Newark = data_Newark[abs(data_Newark['dropoff_longitude'] + 74.190124) <= 0.009999]

data_Newark

In [ ]:
times = pd.DatetimeIndex(data_Newark.pickup_datetime)
grouped_by_hour = data_Newark.groupby([times.hour]).mean()/(60*60)
average_trip_time_Newark = list(grouped_by_hour['trip_duration'])
pickup_hours_Newark = list(grouped_by_hour.index)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(pickup_hours_Newark, average_trip_time_Newark)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Pickup hour',fontsize=15)
plt.ylabel('Average trip duration in hours',fontsize=15)
plt.title('Average trip time from Empire State Building TO Newark Airport against pickup hour',fontsize=15)
plt.show()

### Comments:

The major difference is that there are always trips from Empire State Building to JFK Airport every hour. Whereas, this is not the case from Empire State Building to Newark Airport since some of the averages trip times in the second graph are zeros for several pickup hours. 